Debugging concept

In [3]:
#assert 
import torch

x = torch.randn(3, 4)
print(f"x shape: {x.shape}, values:\n{x}")
y = torch.randn(2, 1)
assert x.shape[0] == y.shape[0], "Batch size mismatch!"


x shape: torch.Size([3, 4]), values:
tensor([[-0.0960,  1.1307, -0.5986,  3.2438],
        [ 0.8856,  0.5584, -0.7827,  0.9766],
        [ 0.1968, -1.1729,  0.6739,  0.6308]])


AssertionError: Batch size mismatch!

In [5]:
#built in debuggers
import pdb; pdb.set_trace()


> c:\users\asus\appdata\local\temp\ipykernel_11852\3064510431.py(2)<module>()



ipdb>  q


In [7]:
#logging
import logging
logging.basicConfig(level=logging.INFO)
logging.info(f"Tensor shape: {x.shape}")


INFO:root:Tensor shape: torch.Size([3, 4])


In [13]:
import torch
import torch.nn as nn

#  model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(4, 3)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = MyModel()


def forward_hook(module, input, output):
    print(f"[FORWARD] {module.__class__.__name__}")
    print(f"  Output shape: {output.shape}")


def backward_hook(module, grad_input, grad_output):
    print(f"[BACKWARD] {module.__class__.__name__}")
    print(f"  grad_output[0] mean: {grad_output[0].mean().item()}")

# Register hooks
fwd_handle = model.fc1.register_forward_hook(forward_hook)
fwd_handle2=model.fc2.register_forward_hook(forward_hook)
bwd_handle = model.fc1.register_full_backward_hook(backward_hook)

# Dummy input & target
x = torch.randn(5, 4)         
target = torch.randn(5, 2)   

# Forward pass
out = model(x)

# Compute loss
loss_fn = nn.MSELoss()
loss = loss_fn(out, target)

# Backward pass
loss.backward()

# Clean up hooks

fwd_handle.remove()
fwd_handle2.remove()
bwd_handle.remove()


[FORWARD] Linear
  Output shape: torch.Size([5, 3])
[FORWARD] Linear
  Output shape: torch.Size([5, 2])
[BACKWARD] Linear
  grad_output[0] mean: -0.005007477477192879


C:\Users\ASUS\AppData\Local\Temp\ipykernel_11852\349787255.py:47: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()


In [14]:
import torch
print(torch.cuda.is_available())   # True if CUDA is usable
print(torch.cuda.device_count())   # Number of GPUs
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


False
0
No GPU
